### Base code

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state

    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

### Problem Code

In [39]:
class Problem_Puzzle(Problem):
    def __init__(self, initial):
        self.__dict__.update(initial=initial, goal=((1, 2, 3), (4, 5, 6), (7, 8, 9)))

    def actions(self, state):
        return [
            "fila 0 derecha",
            "fila 0 izquierda",
            "fila 1 derecha",
            "fila 1 izquierda",
            "fila 2 derecha",
            "fila 2 izquierda",
            "columna 0 abajo",
            "columna 0 arriba",
            "columna 1 abajo",
            "columna 1 arriba",
            "columna 2 abajo",
            "columna 2 arriba",
        ]

    def result(self, state, action):
        real_state = [list(sub_tupla) for sub_tupla in state]

        newState = [row[:] for row in real_state]

        if "derecha" in action:
            fila = int(action.split(" ")[1])
            newState[fila] = [newState[fila][-1]] + newState[fila][:-1]
        elif "izquierda" in action:
            fila = int(action.split(" ")[1])
            newState[fila] = newState[fila][1:] + [newState[fila][0]]
        elif "abajo" in action:
            columna = int(action.split(" ")[1])
            temp = newState[2][columna]
            newState[2][columna] = newState[1][columna]
            newState[1][columna] = newState[0][columna]
            newState[0][columna] = temp
        elif "arriba" in action:
            columna = int(action.split(" ")[1])
            newState[0][columna], newState[1][columna], newState[2][columna] = (
                newState[1][columna],
                newState[2][columna],
                newState[0][columna],
            )

        tupla = tuple(tuple(sub_lista) for sub_lista in newState)
        return tupla

    def h(self, node):
        state = node.state
        goal = self.goal
        m_distance = 0
        disorder = 0
        for i in range(3):
            for j in range(3):
                goal_position = [
                    (index_i, index_j)
                    for index_i, row in enumerate(goal)
                    for index_j, element in enumerate(row)
                    if element == state[i][j]
                ]
                m_distance += abs(i - goal_position[0][0]) + abs(
                    j - goal_position[0][1]
                )

                disorder += sum(
                    1 for k in range(i * 3 + j, 9) if state[k // 3][k % 3] > state[i][j]
                )
        return m_distance + disorder



In [40]:
p1 = Problem_Puzzle(((3, 1, 8), (4, 6, 5), (7, 9, 2)))
p2 = Problem_Puzzle(((9, 8, 7), (6, 5, 4), (3, 2, 1)))
p3 = Problem_Puzzle(((8, 1, 9), (5, 2, 3), (4, 6, 7)))


### Queues

In [4]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

### Search Algorithms

In [42]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result





### Report

#### Report code

In [6]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

#### Run report

In [7]:
print(p1.initial)

((3, 1, 8), (4, 6, 5), (7, 9, 2))


In [44]:
report([uniform_cost_search, breadth_first_bfs], [p1, p2, p3])

uniform_cost_search:
2,177,280 nodes |  181,440 goal |  inf cost | 181,440 actions | Problem_Puzzle(((3, 1, 8), (4, 6, 5), (7
2,173,632 nodes |  181,137 goal |    8 cost | 181,144 actions | Problem_Puzzle(((9, 8, 7), (6, 5, 4), (3
  239,460 nodes |   19,956 goal |    5 cost |  19,960 actions | Problem_Puzzle(((8, 1, 9), (5, 2, 3), (4
4,590,372 nodes |  382,533 goal |  inf cost | 382,544 actions | TOTAL

breadth_first_bfs:
2,177,280 nodes |  181,440 goal |  inf cost | 181,440 actions | Problem_Puzzle(((3, 1, 8), (4, 6, 5), (7
2,173,632 nodes |  181,137 goal |    8 cost | 181,144 actions | Problem_Puzzle(((9, 8, 7), (6, 5, 4), (3
  239,460 nodes |   19,956 goal |    5 cost |  19,960 actions | Problem_Puzzle(((8, 1, 9), (5, 2, 3), (4
4,590,372 nodes |  382,533 goal |  inf cost | 382,544 actions | TOTAL



In [33]:
report([greedy_bfs, astar_search], [p1, p2, p3])

greedy_bfs:
7,064,160 nodes |  588,680 goal |  inf cost | 588,680 actions | Problem_Puzzle(((3, 1, 8), (4, 6, 5), (7
4,593,084 nodes |  382,758 goal |    8 cost | 382,765 actions | Problem_Puzzle(((9, 8, 7), (6, 5, 4), (3
4,599,528 nodes |  383,295 goal |    6 cost | 383,300 actions | Problem_Puzzle(((8, 1, 9), (5, 2, 3), (4
16,256,772 nodes |1,354,733 goal |  inf cost |1,354,745 actions | TOTAL

astar_search:
4,091,808 nodes |  340,984 goal |  inf cost | 340,984 actions | Problem_Puzzle(((3, 1, 8), (4, 6, 5), (7
3,196,092 nodes |  266,342 goal |    8 cost | 266,349 actions | Problem_Puzzle(((9, 8, 7), (6, 5, 4), (3
2,406,264 nodes |  200,523 goal |    5 cost | 200,527 actions | Problem_Puzzle(((8, 1, 9), (5, 2, 3), (4
9,694,164 nodes |  807,849 goal |  inf cost | 807,860 actions | TOTAL



In [45]:
path_actions(astar_search(p2))

['fila 0 derecha',
 'fila 1 izquierda',
 'columna 0 arriba',
 'columna 2 abajo',
 'fila 1 izquierda',
 'fila 0 derecha',
 'columna 1 abajo',
 'columna 2 arriba']

In [46]:
print(p1.h(Node(p1.initial)))

35
